In [16]:
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer
from keras.preprocessing.text import Tokenizer
import nltk
import json
import pickle

#load stop words
#file = open("data/stop_words_spanish",'rb')
#spanish_stopwords = pickle.load(file)
#file.close()

#stemmer = LancasterStemmer()
stemmer = SnowballStemmer('spanish', ignore_stopwords=True)
ignore_words = ['?']

with open('data/stop_words_spanish', 'r') as f:
    spanish_stopwords = [line.strip('\n') for line in f]

In [17]:
# la primera vez que se imprtar NTKL se debe ejecuatr la siguiente setencia para descargar recursos que utiliza dicha biblioteca
# nltk.download('punkt')
#nltk.download('stopwords')

### Funciones para estanadarizar el texto

In [18]:
def normaliza(word):
        # La palabra debe ser en minúsculas
        # elimina tildes 
        # elimina mayúsculas y números
        from unicodedata import normalize, category
        return ''.join([x for x in normalize('NFD', word) if category(x) == 'Ll'])
    
def normalizar_string(cadena):
        #normalizar un cadena (secuencia de palabras)
        result = ""
        # convertir a minúsucla y eliminar espacios en blanco extras
        cadena = cadena.lower().strip()

        for word in cadena.split(" "):
            aux = normaliza(word)
            if len(aux) > 0:
                result = result + ' ' + aux.strip()
            
        return result.strip(' ').strip('\n')
    
def foo(text):
         #eliminar signos de puntuación
        forbidden = {"?", "¿", "¡", "!", " ", ",", ".", ";", ":", "-", "_"}
        return "".join(c for c in text.lower() if c not in forbidden )

In [19]:
def sentence_clean(sentence):
    sentence = normalizar_string(sentence)
    words = nltk.word_tokenize(sentence)
    words_ = []
    [words_.append(w) for w in words if not w in spanish_stopwords]
    words = words_
    del words_

    #stemizar, pasar a minúscula y eliminar duplicadas 
    words = [stemmer.stem(w) for w in words if w not in ignore_words]
    
    
    return ' '.join(words)

### Leer el data set para realizar el pre-procesamiento

In [20]:
import pandas as pd
df = pd.read_csv("dataset_5.csv")

In [21]:
#visulizar los primeros elementos
df.head()

,sentences,class
0,con el contador general,contabilidad
1,con la de contabilidad,contabilidad
2,arrendamientos financieros,contabilidad
3,con el contador general,contabilidad
4,con la de contabilidad,contabilidad


In [22]:
#extraer el contenido del data set como una lista de listas,  sin formato Dataframe
X = df.sentences.values

Transformar la representación de la columna de etiquetas, de  su forma  categórica-texto a categórica-numérica

In [23]:
#convertir clases categoricas 'string' a discretas 'entero'
clases = pd.DataFrame(df['class'].astype('category'))
clases = clases.apply(lambda x: x.cat.codes)
Y = clases.values

In [24]:
Y

array([[ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 2],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 3],
       [ 4],
       [ 4],
       [ 4],
       [ 4],
       [ 4],
       [ 4],
       [ 6],
       [ 6],
       [ 6],
       [ 6],
       [ 7],
       [ 7],
       [ 7],
       [ 7],
       [ 7],
       [ 7],
       [ 7],
       [ 7],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 8],
       [ 8],
       [ 8],
       [ 8],
       [ 9],
       [ 9],
       [ 9],
       [ 9],
       [ 9],
       [ 9],
       [10],
       [10],
       [10],
       [10],
       [11],
       [11],
       [11],
       [11],
       [11],
       [11],
       [11],

#### Texto original 

In [25]:
# create the tokenizer
t = Tokenizer()

# fit the tokenizer on the documents
t.fit_on_texts(X)

# summarize what was learned
print("word_counts: ", t.word_counts)

word_counts:  OrderedDict([('con', 31), ('el', 10), ('contador', 2), ('general', 2), ('la', 6), ('de', 41), ('contabilidad', 2), ('arrendamientos', 2), ('financieros', 2), ('buscando', 2), ('a', 6), ('comunicándome', 3), ('comunicame', 3), ('comunicar', 3), ('comunicarme', 3), ('comunicación', 3), ('buscó', 1), ('conectar', 1), ('extensión', 5), ('está', 1), ('en', 1), ('esta', 1), ('oficina', 1), ('hablar', 3), ('me', 3), ('comunicas', 1), ('comunico', 1), ('trató', 2), ('número', 3), ('se', 1), ('encuentra', 1), ('te', 1), ('transferir', 1), ('un', 4), ('mesaje', 2), ('servicio', 4), ('buzón', 4), ('voz', 4), ('ofimática', 4), ('área', 10), ('informática', 2), ('ofrecer', 2), ('mis', 1), ('productos', 1), ('y', 1), ('servicios', 3), ('llamó', 1), ('empresa', 2), ('ofreciendo', 1), ('los', 1), ('mi', 1), ('revisa', 1), ('correo', 1), ('proveedores', 1), ('referencias', 6), ('laborales', 3), ('recursos', 5), ('humanos', 5), ('una', 2), ('persona', 2), ('que', 2), ('laboró', 1), ('traba

In [26]:
#print("document_count: ",t.document_count)

In [27]:
#print("word_index: ", t.word_index)

In [28]:
#print("word_docs: ",t.word_docs)

### Modelo de bolsa de palabras

In [29]:
#list(X)
#join_sentence = ' '
#X_ = join_sentence.join(X)

X_ = [sentence_clean(x) for x in X]

In [30]:
X_[0:10]

['contador general',
 'contabil',
 'arrend financier',
 'contador general',
 'contabil',
 'arrend financier',
 'busc',
 'comunic',
 '',
 '']

In [31]:
# create the tokenizer
t = Tokenizer()
t.fit_on_texts(X_)

In [32]:
# integer encode documents
encoded_docs = t.texts_to_matrix(X_, mode='binary')
print(encoded_docs)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [33]:
encoded_docs[0:1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [34]:
print(X_[1])
print(encoded_docs[1])
for i,w in enumerate (encoded_docs[10]):
    if w == 1:
        print(i)

contabil
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


otra opción 

### Guardar modelo

In [35]:
#almacenar las stop_words
f = open('stop_words_spanish.pckl', 'wb')
pickle.dump(spanish_stopwords, f)
f.close()


#almacenar el modelo de BoW
f = open('keras_vec.pckl', 'wb')
pickle.dump(t, f)
f.close()

In [36]:

dataset_final = pd.DataFrame(encoded_docs)


In [37]:
dataset_final['class'] = Y
dataset_final['class_cat'] = df['class']
dataset_final.shape

(96, 65)

In [38]:
# dropping  duplicte values 
dataset_final.drop_duplicates(keep = 'first', inplace = True) #subset ="First Name"
dataset_final.shape

(62, 65)

In [40]:
dataset_final.to_csv("sentencias_features.csv",index = False)